# Example: sphere with PS3 dataset

This notebook shows how to use `ffsas` to invert for the radius distribution of a `Sphere` model with a real dataset called "PS3". It uses the [SASView/SASModels](http://www.sasview.org/docs/user/models/sphere.html) unit system.

In [ ]:
# avoid omp error on Mac
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# plotting setup
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 14})
matplotlib.rcParams.update({'legend.fontsize': 14})
matplotlib.rcParams.update({'axes.titlesize': 14})
matplotlib.rcParams.update({'lines.linewidth': 1})
plt.rcParams.update({
    "text.usetex": True,
    "text.latex.preamble":  r'\usepackage{bm,upgreek}',
    "font.family": "sans-serif",
    "font.serif": ["Times"]})
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# create output dir
from pathlib import Path
paper_fig_dir = Path('./output/paper_fig')
Path(paper_fig_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
# uncomment this line to install ffsas
# !pip install ffsas

# ffsas
import torch
import ffsas
from ffsas.models import Sphere
from ffsas.system import SASGreensSystem

# math tools
from scipy import interpolate

# numpy for reading data
import numpy as np

# Read data

Data are stored in the text file `PS3_data/observation.txt`, with the three columns being $q$, mean and standard deviation of the observed intensity.

In [ ]:
# read data
fname = f'PS3_data/observation.txt'
data = np.loadtxt(fname)

# q vector
q = torch.tensor(data[:, 0], dtype=ffsas.torch_dtype)

# intensity mean
mu = torch.tensor(data[:, 1], dtype=ffsas.torch_dtype)

# intensity stddev
sigma = torch.tensor(data[:, 2], dtype=ffsas.torch_dtype)

# Inversion

Just a few lines to do inversion:

In [ ]:
# specify radii
r = torch.linspace(400, 800, 1000)

# compute the Green's tensor
truncate = 285  # truncate noise part at high-q
G = Sphere.compute_G_mini_batch([q[:truncate]], {'r': r}, {'drho': 1.})

# build G-system
g_sys = SASGreensSystem(G, Sphere.get_par_keys_G())

# inversion
# do 1000 iterations and save every 100 iterations
results = g_sys.solve_inverse(mu[:truncate], sigma[:truncate], maxiter=1000, save_iter=100, 
                              trust_options={'xtol': 0, 'gtol':0}, verbose=1)

---

# Plot results

First plot `ffsas` results at different iterations.

In [ ]:
# volume
v = r ** 3

# colormap
cmap = matplotlib.cm.get_cmap('turbo_r')

# steps to plot (every 100 iters)
n_results = len(results['saved_res'])
plot_steps = range(n_results)

# plot
fig=plt.figure(dpi=200, figsize=(7/1.4, 3.5/1.4))
for j, step in enumerate(plot_steps):
    w = results['saved_res'][step]['w_dict']['r']
    w_hat = w * v / (w * v).sum() * 100  # x100 to percent
    plt.plot(r, w_hat, zorder=-j, c=cmap(step / (n_results - 1)),
             label=r'$w(r)$, iters=%d, wct=%.1f sec' % 
             (results['saved_res'][step]['nit'], 
              results['saved_res'][step]['wct']))
    
plt.xlim(r.min(), r.max())
plt.ylabel(r'Volume weight, $\hat{w}_v$ (\%)')
plt.xlabel(r'Radius, $r$ (\AA)')
plt.title(r'(a) Convergence of $\hat{w}_v(r)$ in FFSAS')


# Gaussian approximations of populations
r_ranges=[[420, 500],
          [500, 600],
          [600, 675],
          [675, 800]]
    
for i, (r_min, r_max) in enumerate(r_ranges):
    # find peak
    i_min = torch.argmin(torch.abs(r - r_min))
    i_max = torch.argmin(torch.abs(r - r_max))
    max_loc = torch.argmax(w_hat[i_min:i_max])
    r_top = r[i_min + max_loc]
    
    # find stddev
    area_all = torch.sum(w_hat[i_min:i_max])
    for stddev in range(1, 50):
        area = torch.sum(w_hat[i_min + max_loc - stddev:i_min + max_loc + stddev])
        if area >= area_all * .68:
            break

    # texts
    if r_max == 800:
        plt.text(r_top - 1, w_hat[i_min + max_loc] - 2.2, 
                 r'$\mathcal{N}(%d,%d^2)$' % (round(r_top.item()), 
                                              round(stddev / len(r) * (r.max() - r.min()).item())), 
                 ha='left', va='center', fontsize=13, rotation=45)
    else:
        plt.text(r_top - 20, max(w_hat[i_min + max_loc] + 1., .7), 
                 r'$\mathcal{N}(%d,%d^2)$' % (round(r_top.item()), 
                                              round(stddev / len(r) * (r.max() - r.min()).item())), 
                 ha='left', va='bottom', fontsize=13, rotation=45)
plt.ylim(None, 17)

# legend                 
norm = matplotlib.colors.Normalize(vmin=0, vmax=1000)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbaxes = fig.add_axes([0.18, 0.73, 0.45, .025]) 
cb = plt.colorbar(sm, ticks=np.arange(100,1001,100), 
                  boundaries=np.arange(50,1101,100),
                  cax=cbaxes, orientation='horizontal')
cb.ax.tick_params(labelsize=13) 
cb.ax.set_title('Num. iterations', fontsize=13)
cb.ax.tick_params(axis='x', labelrotation = 45)

# save for paper
plt.savefig(paper_fig_dir / 'PS3a.pdf', bbox_inches='tight', facecolor='w', padding_inches=.05)
plt.show()

Now compare `ffsas`, `McSAS` and `SasView` results. Because the `McSAS` and `SasView` are very smooth, we compare them to an early `ffsas` result (at 30 iterations).

In [ ]:
# inversion at 30 iters
results_early = g_sys.solve_inverse(mu[:truncate], sigma[:truncate], maxiter=30, 
                                    trust_options={'xtol': 0, 'gtol':0}, verbose=1)

In [ ]:
fig=plt.figure(dpi=200, figsize=(7/1.4, 3.5/1.4))

#### FFSAS ####
w_ffsas = results_early['w_dict']['r']
w_hat_ffsas = w_ffsas * v / (w_ffsas * v).sum() * 100
plt.plot(r, w_hat_ffsas, zorder=1000000, lw=1.5, ls='-',
         label=r'FFSAS (%d iters),' % (results_early['opt_res']['nit'],))
plt.plot(r, w_hat_ffsas, zorder=-1000000, lw=1.5, ls='-', c='w',
         label=r'wt=%.1f sec' % (results_early['wct'],))

#### MCSAS ####
# read
data = np.loadtxt('PS3_data/mcsas_w(r).dat', skiprows=1)
vol_frac_low = data[:, 2]

# convert to the same resolution
x = np.linspace(400, 800, len(vol_frac_low))
vol_frac_high = interpolate.interp1d(x, vol_frac_low)(r)

# denoise a little bit to make it look better
nn = 10
vol_frac_denoise = np.convolve(vol_frac_high, np.ones(nn), 'same') / nn
vol_frac_denoise = vol_frac_denoise / vol_frac_denoise.sum() * 100
plt.plot(r, vol_frac_denoise, zorder=1, lw=1.5, ls='--',label=r'McSAS ($\chi^2$-limit),')
plt.plot(r, vol_frac_denoise, zorder=-1000000, lw=1.5, ls='--', c='w', label=r'wt=80.5 sec')


#### SASView ####
# read
r_mean, PD, scale_sasview, b_sasview = np.loadtxt('PS3_data/sasview_gaussian.txt', skiprows=1)
sigm = r_mean * PD

# compute w and w_hat
w_sasview = torch.exp(-.5*((r - r_mean) / sigm) ** 2)
w_sasview = w_sasview / w_sasview.sum()
w_hat_sasview = w_sasview * v / torch.sum(w_sasview * v) * 100

# plot
plt.plot(r, w_hat_sasview, zorder=100, lw=1.5, ls='-.', label='SasView (Gaussian),')
plt.plot(r, w_hat_sasview, zorder=-100000, lw=1.5, c='w', ls='-.', label='wt=1.0 sec')


# figure setting
plt.xlim(400, 800)
plt.ylabel(r'Volume weight, $\hat{w}$ (\%)')
plt.xlabel(r'Radius, $r$ (\AA)')
plt.legend(loc=[0.01, .27], prop={'size': 12}, labelspacing=.3)
plt.title(r"(b) $\hat{w}_v(r)$'s from different methods")

# save for paper
plt.savefig(paper_fig_dir / 'PS3b.pdf', bbox_inches='tight', facecolor='w', padding_inches=.05)
plt.show()

Finally, we compare the intensity fit. First, we compute the predicted intensities.

In [ ]:
# full Green's tensor across q range
G_full = Sphere.compute_G_mini_batch([q], {'r': r}, {'drho': 1.})
g_sys_full = SASGreensSystem(G_full, Sphere.get_par_keys_G())

# FFSAS
I_ffsas = g_sys_full.compute_intensity(w_dict=results_early['w_dict'], 
                                       xi=results['xi'], b=results['b'])

# # McSAS
xi_mcsas, b_mcsas = np.loadtxt('PS3_data/mcsas_xi_b.txt', skiprows=1)
w_mcsas = vol_frac_high / v / (vol_frac_high / v).sum()
I_mcsas = g_sys_full.compute_intensity(w_dict={'r': w_mcsas}, xi=xi_mcsas, b=b_mcsas)

# SasView
xi_sasview = 1e-4 * scale_sasview / (w_sasview * (4 / 3 * np.pi * r ** 3)).sum() 
I_sasview = g_sys_full.compute_intensity(w_dict={'r': w_sasview}, 
                                         xi=xi_sasview, b=b_sasview)

# Monodisperse at 710 A
w_delta = results['w_dict']['r'].clone()
w_delta[:] = 0
w_delta[775]=1  # location of 710 A is 775
# xi and b are found by SasView
scale_delta, b_delta = np.loadtxt('PS3_data/sasview_monodisperse710.txt', skiprows=1)
xi_delta = 1e-4 * scale_delta / (4 / 3 * np.pi * 710 ** 3)
I_delta = g_sys_full.compute_intensity(w_dict={'r': w_delta}, xi=xi_delta, b=b_delta)

In [ ]:
matplotlib.rcParams.update({'font.size': 12})
matplotlib.rcParams.update({'legend.fontsize': 12})
matplotlib.rcParams.update({'axes.titlesize': 12})


fig=plt.figure(dpi=200, figsize=(7/1., 2.52/1.))

# data
plt.errorbar(q, mu, yerr=sigma, c='pink', ecolor='skyblue', lw=1, fmt='o',
             markersize=3, label=r'Observation', zorder=-100)

# FFSAS
I = I_ffsas
eps_norm = np.linalg.norm((mu[:truncate] - I[:truncate]) / sigma[:truncate]) / truncate
plt.plot(q, I, ls='-', lw=2, zorder=0,
         label=r'FFSAS (30 iters),')
plt.plot(q, I, ls='-', lw=2, zorder=-10000000000, c='w',
         label=r'$||\bm{\upepsilon}||$=%.2f' % (eps_norm))    

    
# McSAS
I = I_mcsas
eps_norm = np.linalg.norm((mu[:truncate] - I[:truncate]) / sigma[:truncate]) / truncate
plt.plot(q, I, ls='--', lw=2,
         label=r'McSAS ($\chi^2$-limit),')
plt.plot(q, I, ls='--', lw=2, zorder=-10000000000, c='w',
         label=r'$||\bm{\upepsilon}||$=%.2f' % (eps_norm))

# SASView
I = I_sasview
eps_norm = np.linalg.norm((mu[:truncate] - I[:truncate]) / sigma[:truncate]) / truncate
plt.plot(q, I, ls='-.', lw=2,
         label=r'SasView (Gaussian),' )
plt.plot(q, I, ls='-.', lw=2, zorder=-10000000000, c='w',
         label=r'$||\bm{\upepsilon}||$=%.2f' % (eps_norm))

# monodisperse
I = I_delta
eps_norm = np.linalg.norm((mu[:truncate] - I[:truncate]) / sigma[:truncate]) / truncate
plt.plot(q, I, ls='-', lw=1, zorder=-10,
         label=r'Monodispersity at' )
plt.plot(q, I, ls='-', lw=1, zorder=-10000000000, c='w',
         label=r'$r=710$ \AA, $||\bm{\upepsilon}||$=%.2f' % (eps_norm))

plt.axvline(q[truncate], lw=1.5, c='k', ls=':')
plt.text(q[truncate] / 1.05, 10**4.3, 'Truncation', color='k',
         rotation=90, ha='right', va='top', fontsize=11)

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Scattering vector, $q$ (\AA$^{-1}$)')
plt.ylabel('Intensity, $I$ ($\mathrm{cm}^{-1}$)')
handles, labels = plt.gca().get_legend_handles_labels()
handles.insert(0, handles.pop())
labels.insert(0, labels.pop())
plt.legend(handles, labels, loc=[1.02, 0.0],
           prop={'size': 10})
plt.ylim((10**(-3), 10 ** 4.5))
plt.xlim(q[0] / 1.1, 1.e-1)
plt.title(r"(c) Intensity data and $I(q)$'s from different methods")

# save for paper
plt.savefig(paper_fig_dir / 'PS3c.pdf', bbox_inches='tight', facecolor='w', padding_inches=.05)
plt.show()

---